In [1]:
import pandas as pd

from drugSimilarityTable import SimilarityTable
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem

In [2]:
drugs_csv = pd.read_csv('linked_drugs.csv')

/tmp/ipykernel_120812/3243620423.py:1: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  drugs_csv = pd.read_csv('linked_drugs.csv')


In [3]:
print(type(drugs_csv))

<class 'pandas.core.frame.DataFrame'>


In [4]:
drugs_csv.isnull().sum()

compound-id            0
chembl-id          10883
chembl-name      2054224
smiles                 0
drugbank-id      2084310
drugbank-name    2084310
dtype: int64

In [5]:
drugs_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2095607 entries, 0 to 2095606
Data columns (total 6 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   compound-id    int64 
 1   chembl-id      object
 2   chembl-name    object
 3   smiles         object
 4   drugbank-id    object
 5   drugbank-name  object
dtypes: int64(1), object(5)
memory usage: 95.9+ MB


In [6]:
for drug in drugs_csv.iterrows():
    drug_similarity_table = SimilarityTable(drug[0])
    drug_molecule = Chem.MolFromSmiles(drug[1]['smiles'])
    drug_fingerprint = AllChem.GetMorganFingerprint(drug_molecule, 2)

    for drug_to_test in drugs_csv.iterrows():
        try:
            if drug_to_test[0] != drug[0]:
                drug_to_test_molecule = Chem.MolFromSmiles(drug_to_test[1]['smiles'])
                drug_to_test_fingerprint = AllChem.GetMorganFingerprint(drug_to_test_molecule, 2)
                drugs_similarity = DataStructs.DiceSimilarity(drug_fingerprint, drug_to_test_fingerprint)

                drug_similarity_table.add_item(drug_to_test[0], drugs_similarity)
        except:
            print("Failed Smiles of Drug", drug_to_test[0])
            continue

    drug_similarity_table.print_dictionary()
    break

RDKit WARNING: [11:46:26] WARNING: not removing hydrogen atom without neighbors
[11:46:26] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [11:47:43] WARNING: not removing hydrogen atom without neighbors
[11:47:43] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [11:47:47] WARNING: not removing hydrogen atom without neighbors
[11:47:47] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [11:47:51] WARNING: not removing hydrogen atom without neighbors
[11:47:51] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [11:50:10] WARNING: not removing hydrogen atom without neighbors
[11:50:10] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [11:50:10] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [11:50:10] WARNING: not removing hydrogen atom without neighbors
[11:50:10] WARNING: not removing hydrogen atom without neighbors
[11:50:10] WARNING: not removing hydrogen atom wit

Failed Smiles of Drug 1833727


RDKit ERROR: [11:51:53] Explicit valence for atom # 0 N, 4, is greater than permitted
[11:51:53] Explicit valence for atom # 0 N, 4, is greater than permitted
RDKit ERROR: [11:51:53] Explicit valence for atom # 0 N, 4, is greater than permitted
[11:51:53] Explicit valence for atom # 0 N, 4, is greater than permitted
RDKit ERROR: [11:51:53] Explicit valence for atom # 0 N, 4, is greater than permitted
[11:51:53] Explicit valence for atom # 0 N, 4, is greater than permitted


Failed Smiles of Drug 2085105
Failed Smiles of Drug 2085115
Failed Smiles of Drug 2085513


RDKit ERROR: [11:51:53] Explicit valence for atom # 13 Cl, 5, is greater than permitted
[11:51:53] Explicit valence for atom # 13 Cl, 5, is greater than permitted
RDKit ERROR: [11:51:53] SMILES Parse Error: syntax error while parsing: OS(O)(O)C1=CC=C(C=C1)C-1=C2\C=CC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC=C(C=C1)S(O)(O)O)C1=CC=C(C=C1)S([O-])([O-])[O-])\C1=CC=C(C=C1)S(O)(O)[O-]
[11:51:53] SMILES Parse Error: syntax error while parsing: OS(O)(O)C1=CC=C(C=C1)C-1=C2\C=CC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC=C(C=C1)S(O)(O)O)C1=CC=C(C=C1)S([O-])([O-])[O-])\C1=CC=C(C=C1)S(O)(O)[O-]
RDKit ERROR: [11:51:53] SMILES Parse Error: Failed parsing SMILES 'OS(O)(O)C1=CC=C(C=C1)C-1=C2\C=CC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC=C(C=C1)S(O)(O)O)C1=CC=C(C=C1)S([O-])([O-])[O-])\C1=CC=C(C=C1)S(O)(O)[O-]' for input: 'OS(O)(O)C1=CC=C(C=C1)C-1=C2\C=CC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC=C(C=C1)S(O)(O)O)C1=CC=C(C=C1)S(

...

RDKit ERROR: [11:51:55] Explicit valence for atom # 1 Cl, 4, is greater than permitted
[11:51:55] Explicit valence for atom # 1 Cl, 4, is greater than permitted
RDKit ERROR: [11:51:55] Explicit valence for atom # 0 N, 4, is greater than permitted
[11:51:55] Explicit valence for atom # 0 N, 4, is greater than permitted


Failed Smiles of Drug 2093064
Failed Smiles of Drug 2093616


RDKit ERROR: [11:51:56] Explicit valence for atom # 5 K, 2, is greater than permitted
[11:51:56] Explicit valence for atom # 5 K, 2, is greater than permitted


Failed Smiles of Drug 2094466
33 0.8676470588235294
5 0.8636363636363636
6 0.8529411764705882
57 0.8382352941176471
4 0.835820895522388
59 0.8333333333333334
18 0.8181818181818182
1 0.7883211678832117
7 0.7794117647058824
11 0.7761194029850746


RDKit WARNING: [11:51:56] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [11:51:56] WARNING: not removing hydrogen atom without neighbors
[11:51:56] WARNING: not removing hydrogen atom without neighbors
[11:51:56] WARNING: not removing hydrogen atom without neighbors


In [7]:
for drug in drugs_csv.iterrows():
    drug_similarity_table = SimilarityTable(drug[0])
    drug_molecule = Chem.MolFromSmiles(drug[1]['smiles'])
    drug_fingerprint = AllChem.GetMorganFingerprint(drug_molecule, 4)

    for drug_to_test in drugs_csv.iterrows():
        try:
            if drug_to_test[0] != drug[0]:
                drug_to_test_molecule = Chem.MolFromSmiles(drug_to_test[1]['smiles'])
                drug_to_test_fingerprint = AllChem.GetMorganFingerprint(drug_to_test_molecule, 4)
                drugs_similarity = DataStructs.DiceSimilarity(drug_fingerprint, drug_to_test_fingerprint)

                drug_similarity_table.add_item(drug_to_test[0], drugs_similarity)
        except:
            print("Failed Smiles of Drug", drug_to_test[0])
            continue

    drug_similarity_table.print_dictionary()
    break

RDKit WARNING: [12:31:55] WARNING: not removing hydrogen atom without neighbors
[12:31:55] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [12:33:40] WARNING: not removing hydrogen atom without neighbors
[12:33:40] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [12:33:45] WARNING: not removing hydrogen atom without neighbors
[12:33:45] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [12:33:51] WARNING: not removing hydrogen atom without neighbors
[12:33:51] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [12:37:04] WARNING: not removing hydrogen atom without neighbors
[12:37:04] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [12:37:04] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [12:37:04] WARNING: not removing hydrogen atom without neighbors
[12:37:04] WARNING: not removing hydrogen atom without neighbors
[12:37:04] WARNING: not removing hydrogen atom wit

Failed Smiles of Drug 1833727


RDKit ERROR: [12:39:31] Explicit valence for atom # 0 N, 4, is greater than permitted
[12:39:31] Explicit valence for atom # 0 N, 4, is greater than permitted
RDKit ERROR: [12:39:31] Explicit valence for atom # 0 N, 4, is greater than permitted
[12:39:31] Explicit valence for atom # 0 N, 4, is greater than permitted
RDKit ERROR: [12:39:31] Explicit valence for atom # 0 N, 4, is greater than permitted
[12:39:31] Explicit valence for atom # 0 N, 4, is greater than permitted


Failed Smiles of Drug 2085105
Failed Smiles of Drug 2085115
Failed Smiles of Drug 2085513


RDKit ERROR: [12:39:31] Explicit valence for atom # 13 Cl, 5, is greater than permitted
[12:39:31] Explicit valence for atom # 13 Cl, 5, is greater than permitted
RDKit ERROR: [12:39:31] SMILES Parse Error: syntax error while parsing: OS(O)(O)C1=CC=C(C=C1)C-1=C2\C=CC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC=C(C=C1)S(O)(O)O)C1=CC=C(C=C1)S([O-])([O-])[O-])\C1=CC=C(C=C1)S(O)(O)[O-]
[12:39:31] SMILES Parse Error: syntax error while parsing: OS(O)(O)C1=CC=C(C=C1)C-1=C2\C=CC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC=C(C=C1)S(O)(O)O)C1=CC=C(C=C1)S([O-])([O-])[O-])\C1=CC=C(C=C1)S(O)(O)[O-]
RDKit ERROR: [12:39:31] SMILES Parse Error: Failed parsing SMILES 'OS(O)(O)C1=CC=C(C=C1)C-1=C2\C=CC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC=C(C=C1)S(O)(O)O)C1=CC=C(C=C1)S([O-])([O-])[O-])\C1=CC=C(C=C1)S(O)(O)[O-]' for input: 'OS(O)(O)C1=CC=C(C=C1)C-1=C2\C=CC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC=C(C=C1)S(O)(O)O)C1=CC=C(C=C1)S(

...

RDKit ERROR: [12:39:34] Explicit valence for atom # 0 N, 4, is greater than permitted
[12:39:34] Explicit valence for atom # 0 N, 4, is greater than permitted


Failed Smiles of Drug 2093616


RDKit ERROR: [12:39:35] Explicit valence for atom # 5 K, 2, is greater than permitted
[12:39:35] Explicit valence for atom # 5 K, 2, is greater than permitted


Failed Smiles of Drug 2094466
33 0.8216216216216217
5 0.7821229050279329
57 0.7783783783783784
4 0.7582417582417582
6 0.7243243243243244
1 0.7204301075268817
59 0.6927374301675978
18 0.6666666666666666
7 0.6486486486486487
11 0.6483516483516484


RDKit WARNING: [12:39:35] WARNING: not removing hydrogen atom without neighbors
[12:39:35] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [12:39:35] WARNING: not removing hydrogen atom without neighbors
[12:39:35] WARNING: not removing hydrogen atom without neighbors
